## Read experiment report

In [52]:
import matplotlib.pyplot as plt
import pandas as pd
import os

In [53]:
EXPERIMENT_NAME = "reinforce_log_1_sampling_20_epochs.csv"

In [54]:
df = pd.read_csv(os.path.join("experiments", EXPERIMENT_NAME))
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

## Create additional columns

The `gradient`column represents the current gradient norm. This gradient norm is accumulated across all the steps of a batch.

To estimate the contribution of a given step to the overall gradient, we can do:

```gradient_contribution = current_gradient_norm - previous_gradient_norm```

In [55]:
# Calculate the contribution of each step to the gradient norm 
df['gradient_per_step'] = df.groupby(['epoch', 'batch'])['gradient'].transform(lambda x: x - x.shift().fillna(0))


Each `sampling_epoch` has multiple `epoch`s. The new `overall_epoch` column assings a new overall index to each inner epoch.

In [56]:
inner_epochs = df["epoch"].nunique()
df['overall_epoch'] = df['sampling_epoch'] * inner_epochs + df['epoch'] 

In [ ]:
df['overall_epoch'].value_counts().value_counts()

Add column to identify sex

In [58]:
df["sex"] = df["advantage"].apply(lambda x: "male" if x>0 else "female")

## Check columns

In [ ]:
import matplotlib.pyplot as plt

num_cols = len(df.columns)
num_rows = (num_cols + 2) // 3
fig, axs = plt.subplots(num_rows, 3, figsize=(15, 5*num_rows))
for i, column in enumerate(df.columns):
    if df[column].dtype == 'object':
        df[column].value_counts().plot(kind='bar', ax=axs[i//3, i%3], title=column, alpha=0.7, color='skyblue', edgecolor='black')
        axs[i//3, i%3].set_xlabel('Value')
        axs[i//3, i%3].set_ylabel('Frequency')
        axs[i//3, i%3].grid(True)
    else:
        df[column].plot(kind='hist', ax=axs[i//3, i%3], title=column, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        axs[i//3, i%3].set_xlabel('Value')
        axs[i//3, i%3].set_ylabel('Frequency')
        axs[i//3, i%3].grid(True)

## What is the variance of each step?

In [ ]:
df.groupby(['step'])["log_prob_new"].var().hist(bins=30)

In [ ]:
for step in range(df["step"].min(), df["step"].max()+1, 7):
    step_data = df[df["step"] == step]["log_prob_new"]
    plt.figure(figsize=(5, 5))
    step_data.plot(kind='box', color='skyblue')
    plt.title(f'Log Prob New Box Plot for Step {step}\nVariance: {step_data.var():.4f}')
    plt.ylabel('Value')
    plt.grid(True)
    plt.show()





In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df.groupby(['step'])["log_prob_new"].var(), marker='o', linestyle='-', color='b')
plt.title('Variance of log_prob_new per step')
plt.xlabel('Step')
plt.ylabel('Variance')
plt.grid(True)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df.groupby(['step'])["log_prob_new"].mean(), marker='o', linestyle='-', color='b')
plt.title('Mean of log_prob_new per step')
plt.xlabel('Step')
plt.ylabel('Mean')
plt.grid(True)

The last step exhibits a distinct behavior, characterized by an extremely low variance and a significantly higher probability compared to the other steps. 

Let's dive into this.

In [ ]:
last_step_by_epoch = df[df["step"]==49].groupby("overall_epoch")
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].plot(last_step_by_epoch["log_prob_new"].mean(), marker='o', linestyle='-', color='b')
axs[0].set_title('Mean of log_prob_new per epoch in the last step')
axs[0].set_ylabel('Mean')
axs[0].set_xticks(range(min(last_step_by_epoch.groups.keys()), 1 + max(last_step_by_epoch.groups.keys())))
axs[1].plot(last_step_by_epoch["log_prob_new"].var(), marker='o', linestyle='-', color='b')
axs[1].set_title('Variance of log_prob_new per epoch in the last step')
axs[1].set_ylabel('Variance')
axs[1].set_xticks(range(min(last_step_by_epoch.groups.keys()), 1 + max(last_step_by_epoch.groups.keys())))
for ax in axs:
    ax.grid(True)
plt.tight_layout()
plt.show()

## How are the original probabilites distributed?

In [18]:
per_sample_prob = df.groupby(['overall_epoch', 'batch', 'group', 'sample']).sum()
per_sample_prob.reset_index(inplace=True)

In [ ]:
# Define colors based on reward
colors = ['pink' if "female" in sex else 'blue' for sex in per_sample_prob["sex"]]

# Create scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(per_sample_prob["overall_epoch"], 
            per_sample_prob["log_prob_old"], 
            c=colors, 
            s=50, 
            alpha=0.7, 
            edgecolors='w', 
            linewidth=0.5)

# Add titles and labels
plt.title('Scatter Plot of Log Prob Old vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Log Prob Old')

# Show plot
plt.grid(True)
plt.show()

It seems messy. Maybe I can check if the probabilities are better differentiated in specific steps

In [ ]:
for step in range(df["step"].min(), df["step"].max()+1, 7):
    step_data = df[df["step"] == step]
    
    # Calculate averages for male and female
    female_data = step_data[step_data["sex"].str.contains("female")]
    male_data = step_data[~step_data["sex"].str.contains("female")]
    
    # Create figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3))
    
    # First subplot - Scatter plot with all data
    ax1.scatter(female_data["overall_epoch"], 
               female_data["log_prob_new"], 
               c='pink', 
               s=50, 
               alpha=0.7, 
               edgecolors='w', 
               linewidth=0.5,
               label='Female')
    
    ax1.scatter(male_data["overall_epoch"], 
               male_data["log_prob_new"], 
               c='blue', 
               s=50, 
               alpha=0.7, 
               edgecolors='w', 
               linewidth=0.5,
               label='Male')
    
    # Add titles and labels
    ax1.set_title(f'Log Prob New vs. Epochs (Step {step})')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Log Prob New')
    ax1.legend()
    ax1.grid(True)
    
    # Second subplot - Averages only
    if not female_data.empty:
        # Group by epoch and calculate mean
        female_avg = female_data.groupby("overall_epoch")["log_prob_new"].mean()
        ax2.plot(female_avg.index, female_avg.values, 'pink', linestyle='-', linewidth=2, label='Female Avg')
    
    if not male_data.empty:
        # Group by epoch and calculate mean
        male_avg = male_data.groupby("overall_epoch")["log_prob_new"].mean()
        ax2.plot(male_avg.index, male_avg.values, 'blue', linestyle='-', linewidth=2, label='Male Avg')
    
    # Add titles and labels
    ax2.set_title(f'Average Log Prob New vs. Epochs (Step {step})')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Average Log Prob New')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

Let's check the original probabilities for each sampling epoch

In [ ]:
# Get unique sampling epochs
sampling_epochs = df["sampling_epoch"].unique()

# Get steps to keep
steps = sorted(df["step"].unique())  
steps_to_keep = steps[20:40] 

# Create one plot per sampling epoch
for sampling_epoch in sampling_epochs:
    # Filter data for the current sampling epoch
    df_sampling = df[(df["sampling_epoch"] == sampling_epoch)]
    
    # Group by step and gender to calculate averages
    # Convert to numeric first to avoid TypeError with object dtype
    female_df = df_sampling[df_sampling["sex"] == "female"].copy()
    male_df = df_sampling[df_sampling["sex"] == "male"].copy()
    
    # Ensure log_prob_old is numeric
    female_df["log_prob_old"] = pd.to_numeric(female_df["log_prob_old"], errors='coerce')
    male_df["log_prob_old"] = pd.to_numeric(male_df["log_prob_old"], errors='coerce')
    
    # Now calculate means
    female_avg = female_df.groupby("step")["log_prob_old"].mean()
    male_avg = male_df.groupby("step")["log_prob_old"].mean()
    
    # Filter the averages to include only the selected steps
    female_avg_filtered = female_avg.loc[female_avg.index.intersection(steps_to_keep)]
    male_avg_filtered = male_avg.loc[male_avg.index.intersection(steps_to_keep)]
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    
    if not female_avg_filtered.empty:
        plt.plot(female_avg_filtered.index, female_avg_filtered.values, 'pink', linestyle='-', linewidth=2, label='Female Avg')
    
    if not male_avg_filtered.empty:
        plt.plot(male_avg_filtered.index, male_avg_filtered.values, 'blue', linestyle='-', linewidth=2, label='Male Avg')
    
    # Add titles and labels
    plt.title(f'Average Log Prob Old vs. Step (Sampling Epoch {sampling_epoch}, Steps 20-40)')
    plt.xlabel('Step')
    plt.ylabel('Average Log Prob Old')
    plt.legend()
    plt.grid(True)
    plt.show()


## Loss components

In [ ]:
import matplotlib.pyplot as plt

# Calculate the summed loss_clip for each sample
sample_loss_sums = df.groupby(['sampling_epoch', 'batch', 'group', 'sample'])['loss_clip'].sum().reset_index()

# Create a bar plot for the summed loss_clip for each sample
plt.figure(figsize=(10, 6))
colors = ['pink' if loss < 0 else 'blue' for loss in sample_loss_sums['loss_clip']]
plt.bar(range(len(sample_loss_sums)), sample_loss_sums['loss_clip'], color=colors)

# Add labels and title
plt.title('Summed Loss Clip for Each Sample')
plt.xlabel('Sample Index')
plt.ylabel('Summed Loss Clip')
plt.grid(True, axis='y')

# Add a horizontal line at y=0 for reference
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['log_prob_new'].mean()
avg_pink_old = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['log_prob_old'].mean()
avg_blue = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['log_prob_new'].mean()
avg_blue_old = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['log_prob_old'].mean()
# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Training model')
axs[0].plot(avg_pink.index, avg_pink_old.values, linestyle='--', label='Reference model', color='pink')
axs[0].set_title('Average Log Prob New over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average Log Prob New')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Training model')
axs[1].plot(avg_pink.index, avg_blue_old.values, linestyle='--', label='Reference model', color='blue')
axs[1].set_title('Average Log Prob New over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average Log Prob New')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Average of Women Samples')
axs[2].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Average of Men Samples')
axs[2].set_title('Average Log Prob New over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average Log Prob New')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Create a line plot for each sample within each sampling epoch, epoch, and batch showing the evolution of loss_clip over computed epochs
grouped = per_sample_stats.groupby(['sampling_epoch', 'batch', 'group', 'sample'])
for (sampling_epoch, batch, group, sample), sample_data in grouped:
    plt.figure(figsize=(10, 6))
    color = 'pink' if sample_data['loss_clip'].iloc[0] < 0 else 'blue'
    plt.plot(sample_data["epoch"], sample_data['loss_clip'], marker='o', color=color, label=f'Sampling epoch {sampling_epoch}, Batch {batch}, Group {group}, Sample {sample}')
    plt.title(f'Evolution of Loss Clip over Computed Epochs')
    plt.xlabel('Computed Epochs')
    plt.ylabel('Loss Clip')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
# Create a subplot for each sample showing the evolution of importance_ratio over steps
import matplotlib.pyplot as plt

# Determine the number of subplots needed
num_samples = len(df.groupby(['sampling_epoch', 'batch', 'group', 'sample']))

fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)

# Flatten axes array for easy iteration
axes = axes.flatten()

# Iterate through each group and corresponding subplot
for ax, ((sampling_epoch, batch, group, sample), sample_data) in zip(axes, df.groupby(['sampling_epoch', 'batch', 'group', 'sample'])):
    gender = 'Man' if sample_data['loss_clip'].sum() > 0 else 'Woman'
    for epoch, epoch_data in sample_data.groupby('epoch'):
        ax.plot(epoch_data['step'], epoch_data['importance_ratio'], marker='o', label=f'Epoch {epoch}')
    ax.axhline(y=1-1e-4, color='gray', linestyle='--', linewidth=1, label='Lower Clip (1-1e-4)')
    ax.axhline(y=1+1e-4, color='gray', linestyle='--', linewidth=1, label='Upper Clip (1+1e-4)')
    ax.set_title(f'Sample {sample} / Batch {batch} ({gender})')
    ax.set_xlabel('Steps')
    ax.set_ylabel('Importance Ratio')
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for ax in axes[num_samples:]:
    ax.set_visible(False)
plt.suptitle('Evolution of Importance Ratio over Timesteps')
plt.tight_layout()
plt.show()

In [ ]:
df["log_prob_new"]

In [ ]:
# i want to see the gradient aggregated by sex over the epochs. but the gradient is saved to df accumulated. also it would be interesting to see percentage of clipping. like, for a given epoch, what percentage of the steps
# why the last step has that high probabilty which is the same for all images?
# aggregate gradient (without accumulation) by step,sex, one line per epoch

In [ ]:
# Create a subplot for each sample showing the evolution of log_prob_new over steps
import matplotlib.pyplot as plt

# Determine the number of subplots needed
num_samples = len(df.groupby(['sampling_epoch', 'batch', 'group', 'sample']))

fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)

# Flatten axes array for easy iteration
axes = axes.flatten()

# Iterate through each group and corresponding subplot
for ax, ((sampling_epoch, batch, group, sample), sample_data) in zip(axes, df.groupby(['sampling_epoch', 'batch', 'group', 'sample'])):
    gender = 'Man' if sample_data['loss_clip'].sum() > 0 else 'Woman'
    for epoch, epoch_data in sample_data.groupby('epoch'):
        ax.plot(epoch_data['step'][:-4], epoch_data['log_prob_new'][:-4], marker='o', label=f'Epoch {epoch}')
    ax.axhline(y=1-1e-4, color='gray', linestyle='--', linewidth=1, label='Lower Clip (1-1e-4)')
    ax.axhline(y=1+1e-4, color='gray', linestyle='--', linewidth=1, label='Upper Clip (1+1e-4)')
    ax.set_title(f'Sample {sample} / Batch {batch} ({gender})')
    ax.set_xlabel('Steps')
    ax.set_ylabel('Log Probability New')
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for ax in axes[num_samples:]:
    ax.set_visible(False)
plt.suptitle('Evolution of Log Probability New over Timesteps')
plt.tight_layout()
plt.show()

In [ ]:
# Create a subplot for each sample showing the evolution of gradient over steps
import matplotlib.pyplot as plt

# Determine the number of subplots needed
num_samples = len(df.groupby(['sampling_epoch', 'batch', 'group', 'sample']))

fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)

# Flatten axes array for easy iteration
axes = axes.flatten()

# Iterate through each group and corresponding subplot
for ax, ((sampling_epoch, batch, group, sample), sample_data) in zip(axes, df.groupby(['sampling_epoch', 'batch', 'group', 'sample'])):
    gender = 'Man' if sample_data['loss_clip'].sum() > 0 else 'Woman'
    for epoch, epoch_data in sample_data.groupby('epoch'):
        ax.plot(epoch_data['step'], epoch_data['gradient_per_step'], marker='o', label=f'Epoch {epoch}')
    ax.axhline(y=1-1e-4, color='gray', linestyle='--', linewidth=1, label='Lower Clip (1-1e-4)')
    ax.axhline(y=1+1e-4, color='gray', linestyle='--', linewidth=1, label='Upper Clip (1+1e-4)')
    ax.set_title(f'Sample {sample} / Batch {batch} ({gender})')
    ax.set_xlabel('Steps')
    ax.set_ylabel('Gradient')
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for ax in axes[num_samples:]:
    ax.set_visible(False)
plt.suptitle('Evolution of Gradient over Timesteps')
plt.tight_layout()
plt.show()

In [ ]:
# Create a subplot for each sample showing the evolution of importance_ratio over steps
import matplotlib.pyplot as plt

# Determine the number of subplots needed
num_samples = len(df.groupby(['sampling_epoch', 'batch', 'group', 'sample']))

fig, axes = plt.subplots(2, 5, figsize=(30, 10), sharex=True)

# Flatten axes array for easy iteration
axes = axes.flatten()

# Iterate through each group and corresponding subplot
for ax, ((sampling_epoch, batch, group, sample), sample_data) in zip(axes, df.groupby(['sampling_epoch', 'batch', 'group', 'sample'])):
    gender = 'Man' if sample_data['loss_clip'].sum() > 0 else 'Woman'
    for epoch, epoch_data in sample_data.groupby('epoch'):
        ax.plot(epoch_data['step'], epoch_data['loss_clip'], marker='o', label=f'Epoch {epoch}')
    ax.set_title(f'Sample {sample} / Batch {batch} ({gender})')
    ax.set_xlabel('Steps')
    ax.set_ylabel('Loss clip')
    ax.legend()
    ax.grid(True)

# Hide any unused subplots
for ax in axes[num_samples:]:
    ax.set_visible(False)
plt.suptitle('Evolution of Loss clip over Timesteps')
plt.tight_layout()
plt.show()

In [ ]:
per_sample_stats

In [ ]:
import matplotlib.pyplot as plt

# Create a line plot for each sample within each sampling epoch, epoch, and batch showing the evolution of log_prob_new over computed epochs
grouped = per_sample_stats.groupby(['sampling_epoch', 'batch', 'group', 'sample'])
for (sampling_epoch, batch, group, sample), sample_data in grouped:
    plt.figure(figsize=(10, 6))
    color = 'pink' if sample_data['loss_clip'].iloc[0] < 0 else 'blue'
    plt.plot(sample_data["epoch"], sample_data['kl_loss'], marker='o', color=color, label=f'Sampling epoch {sampling_epoch}, Batch {batch}, Group {group}, Sample {sample}')
    plt.title(f'Evolution of KL loss over Computed Epochs')
    plt.xlabel('Computed Epochs')
    plt.ylabel('KL loss')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
# Calculate and plot the average KL loss of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink_kl = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['kl_loss'].mean()
avg_blue_kl = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['kl_loss'].mean()

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(avg_pink_kl.index, avg_pink_kl.values, marker='o', color='pink', label='Training model')
axs[0].set_title('Average KL Loss over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average KL Loss')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue_kl.index, avg_blue_kl.values, marker='o', color='blue', label='Training model')
axs[1].set_title('Average KL Loss over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average KL Loss')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(avg_pink_kl.index, avg_pink_kl.values, marker='o', color='pink', label='Average of Women Samples')
axs[2].plot(avg_blue_kl.index, avg_blue_kl.values, marker='o', color='blue', label='Average of Men Samples')
axs[2].set_title('Average KL Loss over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average KL Loss')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average gradient per step of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink_grad = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['gradient_per_step'].mean()
avg_blue_grad = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['gradient_per_step'].mean()

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(avg_pink_grad.index, avg_pink_grad.values, marker='o', color='pink', label='Training model')
axs[0].set_title('Average Gradient per Step over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average Gradient per Step')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue_grad.index, avg_blue_grad.values, marker='o', color='blue', label='Training model')
axs[1].set_title('Average Gradient per Step over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average Gradient per Step')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(avg_pink_grad.index, avg_pink_grad.values, marker='o', color='pink', label='Average of Women Samples')
axs[2].plot(avg_blue_grad.index, avg_blue_grad.values, marker='o', color='blue', label='Average of Men Samples')
axs[2].set_title('Average Gradient per Step over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average Gradient per Step')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average KL loss of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink_kl = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['kl_loss'].mean()
avg_blue_kl = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['kl_loss'].mean()

avg_pink_loss_base = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['loss_base'].mean()
avg_blue_loss_base = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['loss_base'].mean()

# Normalize the data
avg_pink_kl = (avg_pink_kl - avg_pink_kl.mean()) / avg_pink_kl.std()
avg_blue_kl = (avg_blue_kl - avg_blue_kl.mean()) / avg_blue_kl.std()
avg_pink_loss_base = (avg_pink_loss_base - avg_pink_loss_base.mean()) / avg_pink_loss_base.std()
avg_blue_loss_base = (avg_blue_loss_base - avg_blue_loss_base.mean()) / avg_blue_loss_base.std()

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(avg_pink_kl.index, avg_pink_kl.values, marker='o', color='pink', label='KL Loss (Women)')
axs[0].plot(avg_pink_loss_base.index, avg_pink_loss_base.values, marker='x', linestyle='--', color='pink', label='Loss Base (Women)')
axs[0].set_title('Average KL Loss over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average KL Loss')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue_kl.index, avg_blue_kl.values, marker='o', color='blue', label='KL Loss (Men)')
axs[1].plot(avg_blue_loss_base.index, avg_blue_loss_base.values, marker='x', linestyle='--', color='blue', label='Loss Base (Men)')
axs[1].set_title('Average KL Loss over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average KL Loss')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(avg_pink_kl.index, avg_pink_kl.values, marker='o', color='pink', label='KL Loss (Women)')
axs[2].plot(avg_blue_kl.index, avg_blue_kl.values, marker='o', color='blue', label='KL Loss (Men)')
axs[2].plot(avg_pink_loss_base.index, avg_pink_loss_base.values, marker='x', linestyle='--', color='pink', label='Loss Base (Women)')
axs[2].plot(avg_blue_loss_base.index, avg_blue_loss_base.values, marker='x', linestyle='--', color='blue', label='Loss Base (Men)')
axs[2].set_title('Average KL Loss over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average KL Loss')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average importance ratio of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink_importance_ratio = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['importance_ratio'].mean()
avg_blue_importance_ratio = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['importance_ratio'].mean()

avg_pink_clipped_ratio = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['clipped_ratio'].mean()
avg_blue_clipped_ratio = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['clipped_ratio'].mean()

# Define min and max lines
min_line = 50 * (1 - 1e-4)
max_line = 50 * (1 + 1e-4)

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(22, 5))

# Plot for pink samples
axs[0].plot(avg_pink_importance_ratio.index, avg_pink_importance_ratio.values, marker='o', color='pink', label='Importance Ratio (Women)')
axs[0].plot(avg_pink_clipped_ratio.index, avg_pink_clipped_ratio.values, marker='x', linestyle='--', color='pink', label='Clipped Ratio (Women)')
axs[0].axhline(y=min_line, color='grey', linestyle='--', label='Min Line')
axs[0].axhline(y=max_line, color='grey', linestyle='--', label='Max Line')
axs[0].set_title('Average Importance Ratio over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average Importance Ratio')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue_importance_ratio.index, avg_blue_importance_ratio.values, marker='o', color='blue', label='Importance Ratio (Men)')
axs[1].plot(avg_blue_clipped_ratio.index, avg_blue_clipped_ratio.values, marker='x', linestyle='--', color='blue', label='Clipped Ratio (Men)')
axs[1].axhline(y=min_line, color='grey', linestyle='--', label='Min Line')
axs[1].axhline(y=max_line, color='grey', linestyle='--', label='Max Line')
axs[1].set_title('Average Importance Ratio over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average Importance Ratio')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(avg_pink_importance_ratio.index, avg_pink_importance_ratio.values, marker='o', color='pink', label='Importance Ratio (Women)')
axs[2].plot(avg_blue_importance_ratio.index, avg_blue_importance_ratio.values, marker='o', color='blue', label='Importance Ratio (Men)')
axs[2].plot(avg_pink_clipped_ratio.index, avg_pink_clipped_ratio.values, marker='x', linestyle='--', color='pink', label='Clipped Ratio (Women)')
axs[2].plot(avg_blue_clipped_ratio.index, avg_blue_clipped_ratio.values, marker='x', linestyle='--', color='blue', label='Clipped Ratio (Men)')
axs[2].axhline(y=min_line, color='grey', linestyle='--', label='Min Line')
axs[2].axhline(y=max_line, color='grey', linestyle='--', label='Max Line')
axs[2].set_title('Average Importance Ratio over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average Importance Ratio')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['clipped_ratio'].mean()
avg_blue = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['clipped_ratio'].mean()
avg_overall = per_sample_stats.groupby('computed_epoch')['clipped_ratio'].mean()

# Create subplots
fig, axs = plt.subplots(2, 2, figsize=(18, 10))

# Plot for pink samples
axs[0, 0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Training model')
axs[0, 0].set_title('Average Clipped Ratio over Computed Epochs (Women Samples)')
axs[0, 0].set_xlabel('Computed Epochs')
axs[0, 0].set_ylabel('Average Clipped Ratio')
axs[0, 0].legend()
axs[0, 0].grid(True)

# Plot for blue samples
axs[0, 1].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Training model')
axs[0, 1].set_title('Average Clipped Ratio over Computed Epochs (Men Samples)')
axs[0, 1].set_xlabel('Computed Epochs')
axs[0, 1].set_ylabel('Average Clipped Ratio')
axs[0, 1].legend()
axs[0, 1].grid(True)

# Plot both pink and blue samples together
axs[1, 0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Average of Women Samples')
axs[1, 0].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Average of Men Samples')
axs[1, 0].set_title('Average Clipped Ratio over Computed Epochs (Women and Men Samples)')
axs[1, 0].set_xlabel('Computed Epochs')
axs[1, 0].set_ylabel('Average Clipped Ratio')
axs[1, 0].legend()
axs[1, 0].grid(True)

# Plot for overall average
axs[1, 1].plot(avg_overall.index, avg_overall.values, marker='o', color='green', label='Overall Average')
axs[1, 1].set_title('Overall Average Clipped Ratio over Computed Epochs')
axs[1, 1].set_xlabel('Computed Epochs')
axs[1, 1].set_ylabel('Overall Average Clipped Ratio')
axs[1, 1].legend()
axs[1, 1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['clipped_ratio'].mean()
avg_blue = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['clipped_ratio'].mean()
avg_overall = per_sample_stats.groupby('computed_epoch')['clipped_ratio'].mean()

# Create subplots
fig, axs = plt.subplots(2, 2, figsize=(18, 10))

# Plot for pink samples
axs[0, 0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Training model')
axs[0, 0].set_title('Average Clipped Ratio over Computed Epochs (Women Samples)')
axs[0, 0].set_xlabel('Computed Epochs')
axs[0, 0].set_ylabel('Average Clipped Ratio')
axs[0, 0].legend()
axs[0, 0].grid(True)

# Plot for blue samples
axs[0, 1].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Training model')
axs[0, 1].set_title('Average Clipped Ratio over Computed Epochs (Men Samples)')
axs[0, 1].set_xlabel('Computed Epochs')
axs[0, 1].set_ylabel('Average Clipped Ratio')
axs[0, 1].legend()
axs[0, 1].grid(True)

# Plot both pink and blue samples together
axs[1, 0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Average of Women Samples')
axs[1, 0].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Average of Men Samples')
axs[1, 0].set_title('Average Clipped Ratio over Computed Epochs (Women and Men Samples)')
axs[1, 0].set_xlabel('Computed Epochs')
axs[1, 0].set_ylabel('Average Clipped Ratio')
axs[1, 0].legend()
axs[1, 0].grid(True)

# Plot for overall average
axs[1, 1].plot(avg_overall.index, avg_overall.values, marker='o', color='green', label='Overall Average')
axs[1, 1].set_title('Overall Average Clipped Ratio over Computed Epochs')
axs[1, 1].set_xlabel('Computed Epochs')
axs[1, 1].set_ylabel('Overall Average Clipped Ratio')
axs[1, 1].legend()
axs[1, 1].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['loss_base'].sum()
avg_blue = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['loss_base'].sum()

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Training model')
axs[0].set_title('Average Loss Base over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average Loss Base')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Training model')
axs[1].set_title('Average Loss Base over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average Loss Base')
axs[1].legend()
axs[1].grid(True)

# Normalize the values so that they can be seen in the same scale
avg_pink_norm = (avg_pink - avg_pink.min()) / (avg_pink.max() - avg_pink.min())
avg_blue_norm = (avg_blue - avg_blue.min()) / (avg_blue.max() - avg_blue.min())

# Plot both pink and blue samples together
axs[2].plot(avg_pink_norm.index, avg_pink_norm.values, marker='o', color='pink', label='Average of Women Samples')
axs[2].plot(avg_blue_norm.index, avg_blue_norm.values, marker='o', color='blue', label='Average of Men Samples')
axs[2].set_title('Average Loss Base over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average Loss Base (Normalized)')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['gradient_per_step'].sum()
avg_blue = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['gradient_per_step'].sum()

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Training model')
axs[0].set_title('Average Gradient over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average Gradient')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Training model')
axs[1].set_title('Average Gradient over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average Gradient')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Average of Women Samples')
axs[2].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Average of Men Samples')
axs[2].plot(avg_blue.index, avg_pink.values+avg_blue.values, marker='o', color='green', label='All Samples')
axs[2].set_title('Average Gradient over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average Gradient')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['log_prob_new'].mean()
avg_blue = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['log_prob_new'].mean()

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Training model')
axs[0].set_title('Average Log Prob New over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average Log Prob New')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Training model')
axs[1].set_title('Average Log Prob New over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average Log Prob New')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Average of Women Samples')
axs[2].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Average of Men Samples')
axs[2].set_title('Average Log Prob New over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average Log Prob New')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()



In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

avg_pink = per_sample_stats[per_sample_stats['loss_clip'] < 0].groupby('computed_epoch')['log_prob_new'].mean()
avg_blue = per_sample_stats[per_sample_stats['loss_clip'] >= 0].groupby('computed_epoch')['log_prob_new'].mean()

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Training model')
axs[0].set_title('Average Log Prob New over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Average Log Prob New')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Training model')
axs[1].set_title('Average Log Prob New over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Average Log Prob New')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(avg_pink.index, avg_pink.values, marker='o', color='pink', label='Average of Women Samples')
axs[2].plot(avg_blue.index, avg_blue.values, marker='o', color='blue', label='Average of Men Samples')
axs[2].set_title('Average Log Prob New over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Average Log Prob New')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
per_sample_stats['computed_epoch'] = per_sample_stats['sampling_epoch'] * 3 + per_sample_stats['epoch'] + per_sample_stats['sampling_epoch']

# Normalize probabilities and log probabilities to range from 0 to 1 for women and men separately
women_samples = per_sample_stats[per_sample_stats['loss_clip'] < 0]
men_samples = per_sample_stats[per_sample_stats['loss_clip'] >= 0]


# Group by computed_epoch and find the sum of log_prob_new and gradient_per_step
sum_pink_gradient = women_samples.groupby('computed_epoch')['gradient_per_step'].sum()
sum_pink_prob = women_samples.groupby('computed_epoch')['log_prob_new'].sum()
sum_blue_gradient = men_samples.groupby('computed_epoch')['gradient_per_step'].sum()
sum_blue_prob = men_samples.groupby('computed_epoch')['log_prob_new'].sum()

# Normalize the sums between 0 and 1
sum_pink_gradient_normalized = (sum_pink_gradient - sum_pink_gradient.min()) / (sum_pink_gradient.max() - sum_pink_gradient.min())
sum_pink_prob_normalized = (sum_pink_prob - sum_pink_prob.min()) / (sum_pink_prob.max() - sum_pink_prob.min())
sum_blue_gradient_normalized = (sum_blue_gradient - sum_blue_gradient.min()) / (sum_blue_gradient.max() - sum_blue_gradient.min())
sum_blue_prob_normalized = (sum_blue_prob - sum_blue_prob.min()) / (sum_blue_prob.max() - sum_blue_prob.min())

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Plot for pink samples
axs[0].plot(sum_pink_gradient_normalized.index, sum_pink_gradient_normalized.values, marker='o', color='pink', label='Max of Gradient for Women Samples', markersize=4)  # Reduced marker size
axs[0].plot(sum_pink_prob_normalized.index, sum_pink_prob_normalized.values, marker='o', color='pink', linestyle='--', label='Max of Log Prob for Women Samples', markersize=4)  # Reduced marker size
axs[0].set_title('Max of Gradient and Log Prob over Computed Epochs (Women Samples)')
axs[0].set_xlabel('Computed Epochs')
axs[0].set_ylabel('Max Value')
axs[0].legend()
axs[0].grid(True)

# Plot for blue samples
axs[1].plot(sum_blue_gradient_normalized.index, sum_blue_gradient_normalized.values, marker='o', color='blue', label='Max of Gradient for Men Samples', markersize=4)  # Reduced marker size
axs[1].plot(sum_blue_prob_normalized.index, sum_blue_prob_normalized.values, marker='o', color='blue', linestyle='--', label='Max of Log Prob for Men Samples', markersize=4)  # Reduced marker size
axs[1].set_title('Max of Gradient and Log Prob over Computed Epochs (Men Samples)')
axs[1].set_xlabel('Computed Epochs')
axs[1].set_ylabel('Max Value')
axs[1].legend()
axs[1].grid(True)

# Plot both pink and blue samples together
axs[2].plot(sum_pink_gradient_normalized.index, sum_pink_gradient_normalized.values, marker='o', color='pink', label='Max of Gradient for Women Samples', markersize=4)  # Reduced marker size
axs[2].plot(sum_blue_gradient_normalized.index, sum_blue_gradient_normalized.values, marker='o', color='blue', label='Max of Gradient for Men Samples', markersize=4)  # Reduced marker size
axs[2].plot(sum_pink_gradient_normalized.index, sum_pink_gradient_normalized.values + sum_blue_gradient_normalized.values, marker='o', color='green', label='Total Max of Gradient', markersize=4)  # Reduced marker size
axs[2].set_title('Max of Gradient over Computed Epochs (Women and Men Samples)')
axs[2].set_xlabel('Computed Epochs')
axs[2].set_ylabel('Max of Gradient')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
df[df["epoch"]==3]

In [ ]:
0.04-1.91

In [ ]:
# Calculate and plot the average of the samples in pink and blue considering the computed epochs
df['computed_epoch'] = df['sampling_epoch'] * 3 + df['epoch'] + df['sampling_epoch']

# Compute number of epochs
n_epochs = len(pd.unique(df["computed_epoch"]))

# Create subplots for each epoch
fig, axs = plt.subplots(n_epochs, 1, figsize=(18, 40))

for epoch in range(n_epochs):
    avg_pink = df[(df['loss_clip'] < 0) & (df['epoch'] == epoch)].groupby(['computed_epoch', 'step'])['gradient_per_step'].mean()
    avg_blue = df[(df['loss_clip'] >= 0) & (df['epoch'] == epoch)].groupby(['computed_epoch', 'step'])['gradient_per_step'].mean()
    avg_total = df[df['epoch'] == epoch].groupby(['computed_epoch', 'step'])['gradient_per_step'].mean()

    # Plot for pink samples
    axs[epoch].plot(avg_pink.index.get_level_values('step'), avg_pink.values, marker='o', color='pink', label='Average of Women Samples')

    # Plot for blue samples
    axs[epoch].plot(avg_blue.index.get_level_values('step'), avg_blue.values, marker='o', color='blue', label='Average of Men Samples')

    # Plot total average
    axs[epoch].plot(avg_total.index.get_level_values('step'), avg_total.values, marker='o', color='green', label='Total Average')

    axs[epoch].set_title(f'Average Gradient over Computed Epochs (Epoch {epoch})')
    axs[epoch].set_xlabel('Step')
    axs[epoch].set_ylabel('Average Gradient')
    axs[epoch].legend()
    axs[epoch].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
(df["loss_base"] + df["loss_clip"]).sum()

In [ ]:
df["kl_loss"].mean()

In [ ]:
df

In [ ]:
df[(df["batch"]==0)&(df["sample"]==2)].groupby("epoch").sum()

In [ ]:
test = df[(df["batch"]==0)&(df["sample"]==2)&(df["epoch"].isin(range(0, 8)))]

# Create a figure with four subplots side by side
fig, axs = plt.subplots(2, 4, figsize=(24, 12))

for i, epoch in enumerate(range(0, 8)):
    importance_ratio = test[test["epoch"] == epoch]["importance_ratio"]
    clipped_ratio = test[test["epoch"] == epoch]["clipped_ratio"]
    sum_importance_ratio = importance_ratio.sum()
    sum_clipped_ratio = clipped_ratio.sum()

    ax = axs[i // 4, i % 4]
    ax.plot(test[test["epoch"] == epoch]["step"], importance_ratio, label='Importance Ratio', marker='o')
    ax.plot(test[test["epoch"] == epoch]["step"], clipped_ratio, label='Clipped Ratio', marker='x')
    ax.set_title(f'Epoch {epoch}')
    ax.set_xlabel('Step')
    ax.set_ylabel('Ratio')
    ax.legend(title=f'Sum Importance: {sum_importance_ratio:.5f}, Sum Clipped: {sum_clipped_ratio:.5f}')
    ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load the image
image_path = 'man.jpg'
image = Image.open(image_path)

# Convert the image to a numpy array
image_array = np.array(image)

# Generate Gaussian noise
mean = 0
stddev = 250  # Standard deviation of the noise
gaussian_noise = np.random.normal(mean, stddev, image_array.shape)

# Add the Gaussian noise to the image
noisy_image_array = image_array + gaussian_noise

# Clip the values to be in the valid range [0, 255]
noisy_image_array = np.clip(noisy_image_array, 0, 255).astype(np.uint8)

# Convert back to an image
noisy_image = Image.fromarray(noisy_image_array)

# Save the noisy image
noisy_image.save('woman_noise.jpeg')

# Display the original and noisy images
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Noisy Image')
plt.imshow(noisy_image)
plt.axis('off')

plt.show()